In [2]:
import math
import random
import cmath
import math
import numpy as np
from collections import deque
import hashlib
import struct

In [ ]:
def load_vector_from_file(filename):
    with open(filename, 'rb') as file:
        # Read the size of the vector (stored as size_t in C++, which is platform-dependent)
        size_data = file.read(struct.calcsize('P'))  
        size = struct.unpack('P', size_data)[0]
        
        # Read the rest of the data as uint64_t values
        vector_data = file.read(size * struct.calcsize('Q')) 
        vector = struct.unpack(f'{size}Q', vector_data)
        
        return list(vector)

In [ ]:
def random_density_function_dna(w, k, function, n_samples=10000, seed = 1):
    sampled_positions = 0
    k_mask = (1 << (2 * k)) - 1  # Mask for k nucleotides (2 bits per nucleotide)

    total_nucleotides = n_samples + w + k - 1  # Total number of nucleotides needed
    random_nucleotides = [random.getrandbits(2) for _ in range(total_nucleotides)]  # Generate random nucleotides

    kmer = 0
    rank_window = deque()
    last_sampled_pos = 0

    # Initialize the rank window with the first w k-mers
    for pos in range(w + k - 1):
        nucleotide = random_nucleotides[pos]  # Get the next nucleotide (2 bits)
        kmer = ((kmer << 2) | nucleotide) & k_mask  # Shift and add nucleotide, mask to keep 2k bits
        if pos >= k - 1:
            rank = function(kmer, w, k, seed)
            rank_window.append(rank)

    # Main loop to process samples
    for sample_idx in range(n_samples):
        rank_window.popleft()

        next_nucleotide = random_nucleotides[w + k - 1 + sample_idx]
        kmer = ((kmer << 2) | next_nucleotide) & k_mask
        suffix_rank = function(kmer, w, k, seed)
        rank_window.append(suffix_rank)

        # Update the last sampled position
        last_sampled_pos -= 1

        # Find the position with the minimum rank
        sampled_pos = rank_window.index(min(rank_window))

        if sampled_pos != last_sampled_pos:
            sampled_positions += 1
            last_sampled_pos = sampled_pos

    total_number_of_kmers = n_samples + w - 1  # Total k-mers processed
    density = sampled_positions / total_number_of_kmers
    return density

# extended from t to k
def random_density_function_dna_extended(w, t, k, function, n_samples=10000, seed = 1):
    sampled_positions = 0
    k_mask = (1 << (2 * k)) - 1  # Mask for k nucleotides (2 bits per nucleotide)

    total_nucleotides = n_samples + w + k - 1  # Total number of nucleotides needed
    random_nucleotides = [random.getrandbits(2) for _ in range(total_nucleotides)]  # Generate random nucleotides

    kmer = 0
    rank_window = deque()
    last_sampled_pos = 0

    # Initialize the rank window with the first w k-mers
    for pos in range(w + k - 1):
        nucleotide = random_nucleotides[pos]  # Get the next nucleotide (2 bits)
        kmer = ((kmer << 2) | nucleotide) & k_mask  # Shift and add nucleotide, mask to keep 2k bits
        if pos >= k - 1:
            # get first t bits of kmer
            tmer = kmer >> (2*k-t)
            # get the other bits
            rest = kmer & ((1 << (2*k-t)) - 1)
            rank = function(tmer, w, t, seed) << (2*k-t) | rest
            rank_window.append(rank)

    # Main loop to process samples
    for sample_idx in range(n_samples):
        rank_window.popleft()

        next_nucleotide = random_nucleotides[w + k - 1 + sample_idx]
        kmer = ((kmer << 2) | next_nucleotide) & k_mask
        tmer=  kmer >> (2*k-t)
        rest = kmer & ((1 << (2*k-t)) - 1)
        suffix_rank = function(tmer, w, t, seed) << (2*k-t) | rest
        rank_window.append(suffix_rank)

        # Update the last sampled position
        last_sampled_pos -= 1

        # Find the position with the minimum rank
        sampled_pos = rank_window.index(min(rank_window))

        if sampled_pos != last_sampled_pos:
            sampled_positions += 1
            last_sampled_pos = sampled_pos

    total_number_of_kmers = n_samples + w - 1  # Total k-mers processed
    density = sampled_positions / total_number_of_kmers
    return density


In [ ]:
def get_gm_minimizer_dna_lex(w,k):
    # Load order
    order = load_vector_from_file(f'gm orders/w{w}_k{k}.bin')

    # get max value
    max_value = 2**k
    rank = max_value
    for i in range(len(order)):
        if order[i] == max_value:
            order[i] = rank
            rank += 1
        



    def gm_minimizer_dna(kmer, w, k, seed):
        original_kmer = kmer  # Save the original kmer for printing
        odd_bits = 0
        even_bits = 0
        bit_position = 0

        for position in range (2*k):
            current_bit = kmer & 1
            if position % 2 == 0:
                even_bits = (current_bit << (position//2)) | even_bits
            else:
                odd_bits = (current_bit << (position//2)) | odd_bits
            kmer >>= 1

        odd_bits_shifted = odd_bits 
        return ((order[odd_bits_shifted] * 2**k)+ even_bits)

    return gm_minimizer_dna

# Loads t but is a minimizer over k 
def get_gm_minimizer_dna_lex_extended(w,t, k):
    # Load order
    order = load_vector_from_file(f'gm orders/w{w}_k{t}.bin')

    # get max value
    max_value = 2**t
    rank = max_value
    for i in range(len(order)):
        if order[i] == max_value:
            order[i] = rank
            rank += 1

    tmer_mask = (1 << t) - 1
    rest_mask = (1 << (k-t)) - 1
        



    def gm_minimizer_dna(kmer, w, k, seed):
        odd_bits = 0
        even_bits = 0

        for position in range (2*k):
            current_bit = kmer & 1
            if position % 2 == 0:
                even_bits = (current_bit << (position//2)) | even_bits
            else:
                odd_bits = (current_bit << (position//2)) | odd_bits
            kmer >>= 1


        tmer = (odd_bits >> (k-t)) & tmer_mask
        rest_mer = odd_bits & rest_mask

        main_rank = order[tmer] << (k-t)
        main_rank + rest_mer

        main_rank_shifted = main_rank << k


        
        return (main_rank_shifted+ even_bits)

    return gm_minimizer_dna

Kraken

In [2]:
w = 17
k = 15
gm_order = get_gm_minimizer_dna_lex(w, k)
random_density_function_dna(w, k, gm_order, 1000000)

# can try generate a minimizer for k = 15 w= 17

0.08771959648645622

Kraken 2

In [3]:
w = 5
k = 31
t = 15

gm_order = get_gm_minimizer_dna_lex_extended(w,t,k)
random_density_function_dna(w,k, gm_order, 100000, 1)

0.2438002479900804

KMC2

In [4]:
prev_w = 15
w = 22
k = 7
gm_order = get_gm_minimizer_dna_lex(prev_w, k)
random_density_function_dna(w, k, gm_order, 100000)

0.07370452205036943

In [5]:
prev_w = 15
w = 49
k = 7
gm_order = get_gm_minimizer_dna_lex(prev_w, k)
random_density_function_dna(w, k, gm_order, 100000)

0.03663241644010875

KMC3

In [6]:
prev_w = 15
w = 16
k = 9
gm_order = get_gm_minimizer_dna_lex(prev_w, k)
random_density_function_dna(w, k, gm_order, 100000)

0.09541568764685297

SSHash

In [3]:
w = 11
k = 21
t = 15

gm_order = get_gm_minimizer_dna_lex_extended(w,t,k)
random_density_function_dna(w,k, gm_order, 100000, 1)

0.12545745425457455

Minimap1/2 / MetaProb2

In [4]:
w = 10
k = 15
gm_order = get_gm_minimizer_dna_lex(w, k)
random_density_function_dna(w, k, gm_order, 1000000)

0.13574877826099566

Giraffe

In [5]:
w = 11
k = 29
t = 15

gm_order = get_gm_minimizer_dna_lex_extended(w,t,k)
random_density_function_dna(w,k, gm_order, 100000, 1)

0.12513748625137486

GraphAligner

In [7]:
previous_w = 15
w = 30
k = 19
t = 15

gm_order = get_gm_minimizer_dna_lex_extended(previous_w,t,k)
random_density_function_dna(w,k, gm_order, 100000, 1)

0.05788321386797829